# Path Following 실험 노트북

## 사진 필요 기능들

### 필요 패키지 삽입

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

# 조이스틱 환경 삽입
import horcrux_terrain_v2
# from horcrux_terrain_v2.envs import PlaneJoyWorld
from horcrux_terrain_v2.envs import PlaneJoyDirWorld

# Ray 패키지 삽입
import ray
import os
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.ppo import PPOConfig

from ray.tune.registry import register_env

import mediapy as media

from scipy.ndimage import uniform_filter1d
from scipy.spatial.transform import Rotation

import matplotlib.pyplot as plt

from gymnasium.utils.save_video import save_video

from IPython.display import Video

### 필요 함수 삽입

In [2]:
def get_unique_filename(base_path, ext=".mp4"):
    """중복된 파일명이 존재하면 숫자를 증가하여 새로운 경로를 반환"""
    if not base_path.endswith(ext):
        base_path += ext  # 확장자 자동 추가

    file_name, file_ext = os.path.splitext(base_path)  # 파일명과 확장자 분리
    count = 0
    new_path = f"{file_name}-episode-0"+file_ext

    while os.path.exists(new_path):  # 파일 존재 여부 확인
        new_path = f"{file_name}{count}-episode-0{file_ext}"
        count += 1


    return f"rl-video{count-1}", new_path


def default_plot(x, y, f_name='default_plot', legends=['acc_x', 'acc_y', 'acc_z'], title=''):
    colors = plt.get_cmap("tab10").colors
    fig, ax = plt.subplots(figsize=(15/2.54, 10/2.54))
    ax.set_facecolor((0.95, 0.95, 0.95)) 

    n_column = len(np.shape(y))
    if n_column>2:
        print("The dimmension of data must be less than 3. (1D or 2D)")
        return -1
    
    n_data = np.shape(y)[1]

    for i in range(n_data):
        # **Plot**
        ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[i], label=legends[i])
        # ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[1], label=legends[1])
        # ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[2], label=legends[2])

    # **Grid 설정**
    ax.grid(True, linestyle="--", linewidth=1, color="#202020", alpha=0.7)  # 주요 그리드
    ax.minorticks_on()
    ax.grid(True, which="minor", linestyle=":", linewidth=0.5, color="#404040", alpha=0.5)  # 보조 그리드

    # **Axis 스타일 설정**
    ax.spines["top"].set_linewidth(1.0)
    ax.spines["right"].set_linewidth(1.0)
    ax.spines["left"].set_linewidth(1.0)
    ax.spines["bottom"].set_linewidth(1.0)

    ax.tick_params(axis="both", labelsize=11, width=1.0)  # 폰트 크기 및 라인 두께
    ax.xaxis.label.set_size(12)
    ax.yaxis.label.set_size(12)

    # **폰트 및 제목 설정**
    plt.rcParams["font.family"] = "Arial"
    ax.set_xlabel("X-Axis", fontsize=12, fontweight="bold")
    ax.set_ylabel("Y-Axis", fontsize=12, fontweight="bold")
    ax.set_title(title, fontsize=14, fontweight="bold")

    # **Legend (MATLAB 스타일 적용)**
    ax.legend(loc="upper right", ncol=3, fontsize=10, frameon=True)

    # **비율 설정 (MATLAB의 `pbaspect([2.1 1 1])`과 비슷한 효과)**
    fig.set_size_inches(2.1 * 5, 5)  # 비율 2.1:1 (기본 높이 5inch 기준)

    # **Save Figure (MATLAB saveas와 유사)**
    plt.savefig(f"./figs/{f_name}.png", dpi=600, bbox_inches="tight")

    plt.show()

def moving_average(data, window_size):
    kernel = np.ones(window_size) / window_size
    return np.convolve(data, kernel, mode='same')  # 'valid'는 경계 제외


def get_data_from_info(info):
    # Action info
    action = np.array([_info['action'] for _info in info])

    # Action direction
    action_direction = np.array([_info['act_direction'] for _info in info])

    # Status info
    stat_init_rpy = np.array([_info['init_rpy'] for _info in info])
    stat_init_com = np.array([_info['init_com'] for _info in info])
    stat_xy_vel = np.array([[_info['x_velocity'], _info['y_velocity']] for _info in info])
    stat_yaw_vel = np.array([_info['yaw_velocity'] for _info in info])
    stat_quat = np.array([_info['head_quat'] for _info in info])
    stat_ang_vel = np.array([_info['head_ang_vel'] for _info in info])
    stat_lin_acc = np.array([_info['head_lin_acc'] for _info in info])
    stat_motion_vector = np.array([_info['motion_vector'] for _info in info])
    stat_com_pos = np.array([_info['com_pos'] for _info in info])
    stat_com_ypr = np.array([_info['com_ypr'] for _info in info])
    stat_step_ypr = np.array([_info['step_ypr'] for _info in info])
    stat_reward_func_orientation = np.array([_info['reward_func_orientation'] for _info in info])
    

    # Rew info
    rew_linear_movement = np.array([_info['reward_linear_movement'] for _info in info])
    reward_angular_movement = np.array([_info['reward_angular_movement'] for _info in info])
    reward_efficiency = np.array([_info['reward_efficiency'] for _info in info])
    reward_healthy = np.array([_info['reward_healthy'] for _info in info])
    cost_ctrl = np.array([_info['cost_ctrl'] for _info in info])
    cost_unhealthy = np.array([_info['cost_unhealthy'] for _info in info])
    cost_orientation = np.array([_info['cost_orientation'] for _info in info])
    cost_yaw_vel = np.array([_info['cost_yaw_vel'] for _info in info])
    cost_proj_dist = np.array([_info['cost_proj_dist'] for _info in info])
    direction_similarity = np.array([_info['direction_similarity'] for _info in info])
    rotation_alignment = np.array([_info['rotation_alignment'] for _info in info])
    vel_orientation = np.array([_info['velocity_theta'] for _info in info])

    # Input info
    input_joy = np.array([_info['joy_input'] for _info in info])
    gait_param = np.array([_info['gait_params'] for _info in info])

    data_dict = {
        'action': action,
        'action_direction': action_direction,
        'stat_init_rpy': stat_init_rpy,
        'stat_init_com': stat_init_com,
        'stat_xy_vel': stat_xy_vel,
        'stat_yaw_vel': stat_yaw_vel,
        'stat_quat': stat_quat,
        'stat_ang_vel': stat_ang_vel,
        'stat_lin_acc': stat_lin_acc,
        'stat_motion_vector': stat_motion_vector,
        'stat_com_pos': stat_com_pos,
        'stat_com_ypr': stat_com_ypr,
        'stat_com_r_ypr':stat_reward_func_orientation,
        'stat_step_ypr': stat_step_ypr,

        'rew_linear_movement': rew_linear_movement,
        'reward_angular_movement': reward_angular_movement,
        'reward_efficiency': reward_efficiency,
        'reward_healthy': reward_healthy,
        'cost_ctrl': cost_ctrl,
        'cost_unhealthy': cost_unhealthy,
        'cost_orientation': cost_orientation,
        'cost_yaw_vel': cost_yaw_vel,
        'cost_proj_dist': cost_proj_dist,
        'direction_similarity': direction_similarity,
        'rotation_alignment': rotation_alignment,
        'vel_orientation': vel_orientation,

        'input_joy': input_joy,
        'gait_param': gait_param,
    }
    
    return data_dict


### Gait 클래스 삽입

In [12]:
import numpy as np

class GaitV2():
    def __init__(self, params:tuple[float, float, int, int, float, int], sampling_t = 0.1, model_timestep = 0.005, frame_skip = 20) -> None:
        self.gait_sampling_interval = model_timestep * frame_skip

        self._ed1 = params[0]
        self._ed2 = params[2]
        self._el1 = params[1]
        self._el2 = params[3]
        self._delta = params[4]

        self._reverse = False
        if len(params) > 5:
            if params[5] > 0:
                self._reverse = False
            else:
                self._reverse = True

        self._t = np.arange(0, 20 * np.pi * (1 / np.gcd(self._ed2, self._el2)), self.gait_sampling_interval).transpose()

        self.MotionMatrix = self.getMotionMat().copy()
        self.joints = self.MotionMatrix.shape[0]
        self.Mvecs = self.MotionMatrix.shape[1]

    def getMotionMat(self)->np.ndarray:
        raw_serp = self.serpenoid(self._t, self._ed1, self._el1, self._ed2, self._el2, self._delta)
        M_mat = self.__genMotionMat(raw_serp)

        return M_mat
    
    def getMvec(self, k):
        if self._reverse:
            k = -k
            
        k = k % self.Mvecs

        return self.MotionMatrix[:,k]
        
    ## Predefined functions
    def serpenoid(self, t, e_d1:float, e_l1:float, e_d2:float, e_l2:float, delta:float)->np.ndarray:
        #Hirose (1993) serpenoid curve implementations
        e_d1 = np.radians(e_d1)
        e_l1 = np.radians(e_l1)
        delta = np.radians(delta)

        f1 = (e_d2/10) * t
        f2 = (e_l2/10) * t

        j_1 = np.sin(e_d1 + f1)
        j_2 = np.sin(e_l1 * 2 + f2 + delta)

        j_3 = np.sin(e_d1 * 3 + f1)
        j_4 = np.sin(e_l1 * 4 + f2 + delta)

        j_5 = np.sin(e_d1 * 5 + f1)
        j_6 = np.sin(e_l1 * 6 + f2 + delta)

        j_7 = np.sin(e_d1 * 7 + f1)
        j_8 = np.sin(e_l1 * 8 + f2 + delta)

        j_9 = np.sin(e_d1 * 9 + f1)
        j_10 = np.sin(e_l1 * 10 + f2 + delta)

        j_11 = np.sin(e_d1 * 11 + f1)
        j_12 = np.sin(e_l1 * 12 + f2 + delta)

        j_13 = np.sin(e_d1 * 13 + f1)
        j_14 = np.sin(e_l1 * 14 + f2 + delta)

        return np.array([j_1, j_2, j_3, j_4, j_5, j_6, j_7, j_8, j_9, j_10, j_11, j_12, j_13, j_14])

    def __genMotionMat(self, serp_pos : np.array)->np.ndarray:
        serp_vel = np.diff(serp_pos.copy()) * (1 / self.gait_sampling_interval)
        serp_tor = np.diff(serp_vel.copy()) * (1 / self.gait_sampling_interval)

        motionMat = np.sign(serp_tor).copy()

        return motionMat

### Path Following 함수 삽입

In [15]:
import math

# 8방향 벡터 (↑, ↗, →, ↘, ↓, ↙, ←, ↖)
DIRECTION_VECTORS = {
    (0, 1): '↑',
    (1, 1): '↗',
    (1, 0): '→',
    (1, -1): '↘',
    (0, -1): '↓',
    (-1, -1): '↙',
    (-1, 0): '←',
    (-1, 1): '↖',
}

def normalize_to_8_direction(dx, dy):
    # 방향 벡터를 정규화하고 8방향 중 가장 가까운 방향으로 매핑
    if dx == 0 and dy == 0:
        return (0, 0)

    octant = 0
    angle = math.atan2(dy, dx) * 180 / math.pi  # 라디안에서 도로 변환

    if angle >= 337.5 or angle < 22.5:
        octant = 2
    elif 22.5 <= angle < 67.5:
        octant = 1
    elif 67.5 <= angle < 112.5:
        octant = 0
    elif 112.5 <= angle < 157.5:
        octant = 7
    elif 157.5 <= angle < 202.5:
        octant = 6
    elif 202.5 <= angle < 247.5:
        octant = 5
    elif 247.5 <= angle < 292.5:
        octant = 4
    elif 292.5 <= angle < 337.5:
        octant = 3
    
    directions = list(DIRECTION_VECTORS.keys())
    return directions[octant]

def get_direction(current_pos, target_pos):
    dx = target_pos[0] - current_pos[0]
    dy = target_pos[1] - current_pos[1]
    direction = normalize_to_8_direction(dx, dy)
    return DIRECTION_VECTORS.get(direction, '·')  # '·'은 정지 상태를 의미

def distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))


## 실행 코드들

### Ray 실행

In [4]:
ray.init(dashboard_host="0.0.0.0", dashboard_port=8265)

2025-06-20 20:00:01,573	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at http://10.130.6.78:8265 


Python version:,3.12.9
Ray version:,2.39.0
Dashboard:,http://10.130.6.78:8265


### Gym 환경 등록

In [5]:
env_config = {
    "forward_reward_weight": 175.0,
    "rotation_reward_weight": 100.0,
    "unhealthy_max_steps": 150.0,
    "healthy_reward": 3.0,
    "healthy_roll_range": (-40,40),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 14.5,
    "termination_reward": 0,
    "gait_params": (30, 30, 40, 40, 0, -1),
    "use_friction_chg": True,
    "joy_input_random": True,
    "use_imu_window": True,
    "use_vels_window": True,
    "ctrl_cost_weight": 0.05,
}

render_env_config = env_config.copy()
render_env_config['render_mode'] = 'rgb_array'
render_env_config['render_camera_name'] = 'ceiling'

# env = gym.make("horcrux_terrain_v2/plane-v2", **render_env_config)

# JoyWorld
register_env("joy-v1", lambda config: PlaneJoyDirWorld( forward_reward_weight=env_config["forward_reward_weight"], 
                                                     rotation_reward_weight=env_config["rotation_reward_weight"], 
                                                     unhealthy_max_steps=env_config["unhealthy_max_steps"],
                                                     healthy_reward=env_config["healthy_reward"], 
                                                     healthy_roll_range=env_config["healthy_roll_range"],
                                                     terminating_roll_range=env_config["terminating_roll_range"],
                                                     rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                     termination_reward=env_config["termination_reward"],
                                                     gait_params=env_config["gait_params"],
                                                     use_friction_chg=env_config["use_friction_chg"],
                                                     joy_input_random=env_config["joy_input_random"],
                                                     use_imu_window=env_config["use_imu_window"],
                                                     ctrl_cost_weight=env_config["ctrl_cost_weight"],
                                                   )
            )

### 각 방향 정책 로드

In [ ]:
# base_path = '/home/bong/Project/snake_RL/GD_tor/learning/ray239-ppo/selected_policy/'

# ALGO_FORWARD = Algorithm.from_checkpoint(path=base_path + 'FORW/')
# ALGO_BACKWARD = Algorithm.from_checkpoint(path=base_path + 'BACK/')
# ALGO_LEFT = Algorithm.from_checkpoint(path=base_path + 'UP/')
# ALGO_RIGHT = Algorithm.from_checkpoint(path=base_path + 'DOWN/')
# ALGO_FORWARD_LEFT = Algorithm.from_checkpoint(path=base_path + '45DEG/')
# ALGO_FORWARD_RIGHT = Algorithm.from_checkpoint(path=base_path + '315DEG/')
# ALGO_BACKWARD_LEFT = Algorithm.from_checkpoint(path=base_path + '135DEG/')
# ALGO_BACKWARD_RIGHT = Algorithm.from_checkpoint(path=base_path + '225DEG/')

base_path = '/home/bong/Project/snake_RL/GD_tor/learning/ray239-ppo/selected_policy/LearnerPolicy/'

ALGO_FORWARD = Algorithm.from_checkpoint(path=base_path + 'ForwardPolicy/')
ALGO_BACKWARD = Algorithm.from_checkpoint(path=base_path + 'BackPolicy/')
ALGO_LEFT = Algorithm.from_checkpoint(path=base_path + 'UpPolicy/')
ALGO_RIGHT = Algorithm.from_checkpoint(path=base_path + 'DownPolicy/')
ALGO_FORWARD_LEFT = Algorithm.from_checkpoint(path=base_path + '45DegPolicy/')
ALGO_FORWARD_RIGHT = Algorithm.from_checkpoint(path=base_path + '315DegPolicy/')
ALGO_BACKWARD_LEFT = Algorithm.from_checkpoint(path=base_path + '135DegPolicy/')
ALGO_BACKWARD_RIGHT = Algorithm.from_checkpoint(path=base_path + '225DegPolicy/')


/home/bong/anaconda3/envs/gd239/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:567: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/bong/anaconda3/envs/gd239/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/bong/anaconda3/envs/gd239/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is depreca

### 경로추종

In [16]:
from pprint import pprint
import datetime
from scipy.io import savemat

# Joystick 값 리스트
# 8방향 정의 (정규화된 단위 벡터)
# directions = [
    # np.array([0, 1]),                                # ↑ 30,30,40,40,45
    # np.array([np.sqrt(2)/2, np.sqrt(2)/2]),          # ↗ 30,30,40,40,180
    # np.array([1, 0]),                                # → 30,30,40,40,0
    # np.array([np.sqrt(2)/2, -np.sqrt(2)/2]),         # ↘ 30,30,40,40,0
    # np.array([0, -1]),                               # ↓ 30,30,40,40,315
    # np.array([-np.sqrt(2)/2, -np.sqrt(2)/2]),        # ↙ 30,30,40,40,45,-1
    # np.array([-1, 0]),                               # ← 30,30,40,40,180,-1
    # np.array([-np.sqrt(2)/2, np.sqrt(2)/2])          # ↖ 30,30,40,40,315,-1
# ]

# gait_params =[
    # (30, 30, 40, 40, 45),   # ↑
    # (30, 30, 40, 40, 23),  # ↗
    # (30, 30, 40, 40, 0),  # →
    # (30, 30, 40, 40, -23),    # ↘
    # (30, 30, 40, 40, -45),  # ↓
    # (30, 30, 40, 40, 23, -1),   # ↙
    # (30, 30, 40, 40, 0, -1),    # ←
    # (30, 30, 40, 40, -23, -1)   # ↖
# ]

FORWARD_GAIT = GaitV2((30, 30, 40, 40, 0), sampling_t = 0.05, frame_skip=10)
FORWARD_LEFT_GAIT = GaitV2((30, 30, 40, 40, 23), sampling_t = 0.05, frame_skip=10)
LEFT_GAIT = GaitV2((30, 30, 40, 40, 45), sampling_t = 0.05, frame_skip=10)
BACKWARD_LEFT_GAIT = GaitV2((30, 30, 40, 40, -23, -1), sampling_t = 0.05, frame_skip=10)
BACKWARD_GAIT = GaitV2((30, 30, 40, 40, 0, -1), sampling_t = 0.05, frame_skip=10)
BACKWARD_RIGHT_GAIT = GaitV2((30, 30, 40, 40, 23, -1), sampling_t = 0.05, frame_skip=10)
RIGHT_GAIT = GaitV2((30, 30, 40, 40, -45), sampling_t = 0.05, frame_skip=10)
FORWARD_RIGHT_GAIT = GaitV2((30, 30, 40, 40, -23), sampling_t = 0.05, frame_skip=10)



def follow_path(start_pos, path, epsilon=0.5):
    k = 0
    config = ALGO_FORWARD.get_config()
    save_name = "PathFollowing_8dir_"

    env = gym.make("horcrux_terrain_v2/plane-v3", **render_env_config)
    obs = env.reset()[0]
    _motion_vector = FORWARD_GAIT.getMvec(k).copy()
    obs[-17:-3] = _motion_vector.copy()
    obs[-3:] = np.array([1, 0, 0])

    rew_return = 0
    frames = []
    info = []
    current_pos = list(start_pos)
    path_index = 0

    # while path_index < len(path):
    for _ in range(3000):
        target = path[path_index]
        dist = distance(current_pos, target)

        if dist < epsilon:
            print(f"✅ 도달: {target}")
            path_index += 1
            continue

        direction = get_direction(current_pos, target)
        print(f"👉 현재 위치: {current_pos}, 목표: {target}, 방향 입력: {direction}, 목표와 거리: {dist}")

        if direction == '↑':
            _motion_vector = LEFT_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([0, 1, 0])
            action = ALGO_LEFT.compute_single_action(obs, explore=False)
        elif direction == '↓':
            _motion_vector = RIGHT_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([0, -1, 0])
            action = ALGO_RIGHT.compute_single_action(obs, explore=False)
        elif direction == '←':
            _motion_vector = BACKWARD_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([-1, 0, 0])
            action = ALGO_BACKWARD.compute_single_action(obs, explore=False)
        elif direction == '→':
            _motion_vector = FORWARD_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([1, 0, 0])
            action = ALGO_FORWARD.compute_single_action(obs, explore=False)
        elif direction == '↗':
            _motion_vector = FORWARD_LEFT_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([np.sqrt(2)/2, np.sqrt(2)/2, 0])
            action = ALGO_FORWARD_LEFT.compute_single_action(obs, explore=False)
        elif direction == '↘':
            _motion_vector = FORWARD_RIGHT_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([np.sqrt(2)/2, -np.sqrt(2)/2, 0])
            action = ALGO_FORWARD_RIGHT.compute_single_action(obs, explore=False)
        elif direction == '↙':
            _motion_vector = BACKWARD_RIGHT_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([-np.sqrt(2)/2, -np.sqrt(2)/2, 0])
            action = ALGO_BACKWARD_RIGHT.compute_single_action(obs, explore=False)
        elif direction == '↖':
            _motion_vector = BACKWARD_LEFT_GAIT.getMvec(k).copy()
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([-np.sqrt(2)/2, np.sqrt(2)/2, 0])
            action = ALGO_BACKWARD_LEFT.compute_single_action(obs, explore=False)
        else:
            _motion_vector = np.zeros(14)
            obs[-17:-3] = _motion_vector
            obs[-3:] = np.array([0, 0, 0])
            action = np.array([0]*14)

        obs, _step_rew, _, env_done, env_info = env.step(action)
        env_info['act_direction'] = direction

        pixels = env.render()
        frames.append(pixels)
        info.append(env_info)
        rew_return += _step_rew

        current_pos = env_info['com_pos'][:2]  # 현재 위치 업데이트
        k += 1 # 게이트 벡터 인덱스 증가

    _video_base_name = 'rl-video'

    _f_name, _full_path = get_unique_filename(f"./thesis/{_video_base_name}")
    rew_dict = get_data_from_info(info)
    rew_dict['rew_return'] = rew_return
    rew_dict['motionMatrix'] = info[-1]['motionMatrix']

    # Save Video
    save_video(frames, "./thesis/", name_prefix=_f_name, fps=env.metadata['render_fps'])

    # Save Reward Info mat file
    savemat(f"./thesis/{save_name}_{_f_name}.mat", rew_dict)

    env.reset()
    env.close()



# 예시 경로
path = [(0, 0), (2, 2), (4, 0), (6, 3)]
start_position = (-0.4795, 0)

# 경로 추종 시작
follow_path(start_position, path, epsilon=0.5)



✅ 도달: (0, 0)
👉 현재 위치: [-0.4795, 0], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.185580049221805
👉 현재 위치: [-0.4795  0.    ], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.185580049221805
👉 현재 위치: [-4.79499885e-01  3.55549504e-06], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.185577727612747
👉 현재 위치: [-0.47963398 -0.00226025], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.1871038240645198
👉 현재 위치: [-0.47729341 -0.00617796], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.1877472671892004
👉 현재 위치: [-0.47355693 -0.01237595], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.188752237658986
👉 현재 위치: [-0.47123989 -0.02609779], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.195637468541989
👉 현재 위치: [-0.4657167  -0.04114261], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.200940800748377
👉 현재 위치: [-0.46066341 -0.06122908], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.209911166789161
👉 현재 위치: [-0.45438472 -0.08073491], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.217679613799409
👉 현재 위치: [-0.45051504 -0.09280833], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.222556546487799
👉 현재 위치: [-0.45061153 -0.10155897], 목표: (2, 2), 방향 입력: ↗, 목표와 거리: 3.228319